In [1]:
#r "nuget: Microsoft.Extensions.Configuration"
#r "nuget: Microsoft.Extensions.Configuration.Json"
#r "nuget: Neo4j.Driver"

using Neo4j.Driver;
using System.Text.Json;


Installed Packages Microsoft.Extensions.Configuration, 8.0.0 Microsoft.Extensions.Configuration.Json, 8.0.0 Neo4j.Driver, 5.18.1

In [2]:
#!import config/Config.cs

In [3]:
Console.WriteLine($"Neo4j Uri {Config.Neo4jUri}");
Console.WriteLine($"Neo4j User {Config.Neo4jUser}");
Console.WriteLine($"Neo4j Password {Config.Neo4jPassword}");

Neo4j Uri neo4j://localhost:7687
Neo4j User neo4j
Neo4j Password marwhompa


In [4]:
var driver = GraphDatabase.Driver(Config.Neo4jUri, AuthTokens.Basic(Config.Neo4jUser, Config.Neo4jPassword));
await driver.VerifyConnectivityAsync();

## Working with Cypher Results

### hello world!

The simplest Cypher query is a `RETURN` clause that returns a string literal. This query returns the string "Hello, World!".

Notice that the `Result` below contains two parts:

1. `Result` showing the data returned by the query.
2. `Summary` showing information about the query execution.

In [5]:
driver.ExecutableQuery("RETURN 'hello world' as message").ExecuteAsync().Result

Neo4j.Driver.EagerResult`1[System.Collections.Generic.IReadOnlyList`1[Neo4j.Driver.IRecord]] Keys [ message ] Result index value 0 [ [message, hello world] ] Keys [ message ] Values [ [message, hello world] ] Keys [ message ] Values [ [message, hello world] ] Keys [ message ] Values [ [message, hello world] ] Keys [ message ] Values [ [message, hello world] ] (values) index value 0 [message, hello world] (values) index value 0 [message, hello world] Key message Value hello world (values) index value 0 [message, hello world] Key message Value hello world (values) index value 0 [message, hello world] Key message Value hello world Summary ResultSummary{Query=`RETURN 'hello world' as message`, [], Counters=Counters{NodesCreated=0, NodesDeleted=0, RelationshipsCreated=0, RelationshipsDeleted=0, PropertiesSet=0, LabelsAdded=0, LabelsRemoved=0, IndexesAdded=0, IndexesRemoved=0, ConstraintsAdded=0, ConstraintsRemoved=0, SystemUpdates=0}, ... Query `RETURN 'hello world' as message`, [] Text RETURN 'hello world' as message Parameters (empty) Counters Counters{NodesCreated=0, NodesDeleted=0, RelationshipsCreated=0, RelationshipsDeleted=0, PropertiesSet=0, LabelsAdded=0, LabelsRemoved=0, IndexesAdded=0, IndexesRemoved=0, ConstraintsAdded=0, ConstraintsRemoved=0, SystemUpdates=0} ContainsUpdates False NodesCreated 0 NodesDeleted 0 RelationshipsCreated 0 RelationshipsDeleted 0 PropertiesSet 0 LabelsAdded 0 LabelsRemoved 0 IndexesAdded 0 IndexesRemoved 0 ConstraintsAdded 0 ConstraintsRemoved 0 SystemUpdates 0 ContainsSystemUpdates False QueryType ReadOnly HasPlan False HasProfile False Plan <null> Profile <null> Notifications <null> ResultAvailableAfter 00:00:00.0050000 ResultConsumedAfter 00:00:00.0010000 Server ServerInfo{Address=localhost:7687, Agent=Neo4j/5.16.0, ProtocolVersion=5.4} ProtocolVersion 5.4 Agent Neo4j/5.16.0 Address localhost:7687 Database Neo4j.Driver.Internal.Result.DatabaseInfo Name neo4j

### Get specific result value

The `Result.Result` is a list of IRecord objects. Each IRecord object contains a list of key-value pairs. You can access the value of a specific key by using either:

1. `<T>Get(string key)` to cast the value to a specific type.
2. `<T>As<T>(string key)` to cast the value to a specific type.

In [6]:
var gotMessage = driver.ExecutableQuery("RETURN 'hello world' as message").ExecuteAsync().Result.Result[0].Get<string>("message");
var gotField = driver.ExecutableQuery("RETURN 'hello world' as message").ExecuteAsync().Result.Result[0]["message"];
var gotFieldAs = driver.ExecutableQuery("RETURN 'hello world' as message").ExecuteAsync().Result.Result[0]["message"].As<string>();

Console.WriteLine($"Got message: {gotMessage}");
Console.WriteLine($"Got field: {gotField}");
Console.WriteLine($"Got field as: {gotFieldAs}");

Got message: hello world
Got field: hello world
Got field as: hello world


### map each result row

Add `WithMap()` to apply a map function to each result row. 

Here, we're converting each row from a record to a simple string...

In [7]:
driver.ExecutableQuery("RETURN 'hello world' as message")
  .WithMap(row => row["message"].As<string>())
  .ExecuteAsync().Result.Result[0]

hello world

## Create data

### Create a Node

As graph query language, Cypher works with patterns of data. Patterns are used to create and find data.

The simplest pattern is a single "node", which is what data records are called in a graph database.

The following query creates a single node, assigning the variable `a` to that node and returning the `a` node.

In [21]:
driver.ExecutableQuery("CREATE (a) RETURN a").ExecuteAsync().Result.Result

index value 0 [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 5 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:5 Labels [ ] Properties [ ] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 5 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:5 Labels [ ] Properties (empty) (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 5 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:5 Labels [ ] Properties (empty)

### Create a Node with a Label

Nodes can belong to one or more sets identified by "label". You can set a label when creating a node using the
notation `(:NameOfLabel)`.

For example, the following query creates a node with the label `Person`.

In [20]:
driver.ExecutableQuery("CREATE (a:Person) RETURN a").ExecuteAsync().Result.Result

index value 0 [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 4 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:4 Labels [ Person ] Properties [ ] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 4 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:4 Labels [ Person ] Properties (empty) (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 4 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:4 Labels [ Person ] Properties (empty)

### Create a Node with Properties

As data records, nodes can also carry properties. Properties are key-value pairs.

The following query creates a node with the label `Person` and two properties: `name` and `height`.

In [19]:
driver.ExecutableQuery("CREATE (a:Person {name:'Andreas', height: 186}) RETURN a").ExecuteAsync().Result.Result

index value 0 [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 3 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:3 Labels [ Person ] Properties [ [name, Andreas], [height, 186] ] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 3 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:3 Labels [ Person ] Properties key type value name System.String Andreas height System.Int64 186 (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 3 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:3 Labels [ Person ] Properties key type value name System.String Andreas height System.Int64 186

## Find data

Finding or reading data also uses patterns. You can use similar single-node patterns.

### Find all Nodes

The following query finds all nodes in the graph by using the `MATCH` clause with a single node pattern.

Like `CREATE`, the `MATCH` clause binds variables within a pattern which can be used for selecting results.

Here, the `all` variable will match any node within the graph, producing as many result rows as there are nodes. For larger graphs, this is probably a bad idea.

In [18]:
driver.ExecutableQuery("MATCH (all) RETURN all").ExecuteAsync().Result.Result

index value 0 [ [all, Neo4j.Driver.Internal.Types.Node] ] Keys [ all ] Values [ [all, Neo4j.Driver.Internal.Types.Node] ] Keys [ all ] Values [ [all, Neo4j.Driver.Internal.Types.Node] ] Keys [ all ] Values [ [all, Neo4j.Driver.Internal.Types.Node] ] Keys [ all ] Values [ [all, Neo4j.Driver.Internal.Types.Node] ] Keys [ all ] Values [ [all, Neo4j.Driver.Internal.Types.Node] ] (values) index value 0 [all, Neo4j.Driver.Internal.Types.Node] (values) index value 0 [all, Neo4j.Driver.Internal.Types.Node] Key all Value Neo4j.Driver.Internal.Types.Node (values) index value 0 [all, Neo4j.Driver.Internal.Types.Node] Key all Value Neo4j.Driver.Internal.Types.Node Id 0 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:0 Labels [ ] Properties [ ] (values) index value 0 [all, Neo4j.Driver.Internal.Types.Node] Key all Value Neo4j.Driver.Internal.Types.Node Id 0 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:0 Labels [ ] Properties (empty) (values) index value 0 [all, Neo4j.Driver.Internal.Types.Node] Key all Value Neo4j.Driver.Internal.Types.Node Id 0 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:0 Labels [ ] Properties (empty) 1 [ [all, Neo4j.Driver.Internal.Types.Node] ] Keys [ all ] Values [ [all, Neo4j.Driver.Internal.Types.Node] ] Keys [ all ] Values [ [all, Neo4j.Driver.Internal.Types.Node] ] Keys [ all ] Values [ [all, Neo4j.Driver.Internal.Types.Node] ] Keys [ all ] Values [ [all, Neo4j.Driver.Internal.Types.Node] ] Keys [ all ] Values [ [all, Neo4j.Driver.Internal.Types.Node] ] (values) index value 0 [all, Neo4j.Driver.Internal.Types.Node] (values) index value 0 [all, Neo4j.Driver.Internal.Types.Node] Key all Value Neo4j.Driver.Internal.Types.Node (values) index value 0 [all, Neo4j.Driver.Internal.Types.Node] Key all Value Neo4j.Driver.Internal.Types.Node Id 1 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:1 Labels [ Person ] Properties [ ] (values) index value 0 [all, Neo4j.Driver.Internal.Types.Node] Key all Value Neo4j.Driver.Internal.Types.Node Id 1 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:1 Labels [ Person ] Properties (empty) (values) index value 0 [all, Neo4j.Driver.Internal.Types.Node] Key all Value Neo4j.Driver.Internal.Types.Node Id 1 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:1 Labels [ Person ] Properties (empty) 2 [ [all, Neo4j.Driver.Internal.Types.Node] ] Keys [ all ] Values [ [all, Neo4j.Driver.Internal.Types.Node] ] Keys [ all ] Values [ [all, Neo4j.Driver.Internal.Types.Node] ] Keys [ all ] Values [ [all, Neo4j.Driver.Internal.Types.Node] ] Keys [ all ] Values [ [all, Neo4j.Driver.Internal.Types.Node] ] Keys [ all ] Values [ [all, Neo4j.Driver.Internal.Types.Node] ] (values) index value 0 [all, Neo4j.Driver.Internal.Types.Node] (values) index value 0 [all, Neo4j.Driver.Internal.Types.Node] Key all Value Neo4j.Driver.Internal.Types.Node (values) index value 0 [all, Neo4j.Driver.Internal.Types.Node] Key all Value Neo4j.Driver.Internal.Types.Node Id 2 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:2 Labels [ Person ] Properties [ [name, Andreas], [height, 186] ] (values) index value 0 [all, Neo4j.Driver.Internal.Types.Node] Key all Value Neo4j.Driver.Internal.Types.Node Id 2 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:2 Labels [ Person ] Properties key type value name System.String Andreas height System.Int64 186 (values) index value 0 [all, Neo4j.Driver.Internal.Types.Node] Key all Value Neo4j.Driver.Internal.Types.Node Id 2 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:2 Labels [ Person ] Properties key type value name System.String Andreas height System.Int64 186

### Find Nodes with a Label

You can filter nodes by label by adding the label to the node pattern.

In [17]:
driver.ExecutableQuery("MATCH (p:Person) RETURN p").ExecuteAsync().Result.Result

index value 0 [ [p, Neo4j.Driver.Internal.Types.Node] ] Keys [ p ] Values [ [p, Neo4j.Driver.Internal.Types.Node] ] Keys [ p ] Values [ [p, Neo4j.Driver.Internal.Types.Node] ] Keys [ p ] Values [ [p, Neo4j.Driver.Internal.Types.Node] ] Keys [ p ] Values [ [p, Neo4j.Driver.Internal.Types.Node] ] Keys [ p ] Values [ [p, Neo4j.Driver.Internal.Types.Node] ] (values) index value 0 [p, Neo4j.Driver.Internal.Types.Node] (values) index value 0 [p, Neo4j.Driver.Internal.Types.Node] Key p Value Neo4j.Driver.Internal.Types.Node (values) index value 0 [p, Neo4j.Driver.Internal.Types.Node] Key p Value Neo4j.Driver.Internal.Types.Node Id 1 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:1 Labels [ Person ] Properties [ ] (values) index value 0 [p, Neo4j.Driver.Internal.Types.Node] Key p Value Neo4j.Driver.Internal.Types.Node Id 1 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:1 Labels [ Person ] Properties (empty) (values) index value 0 [p, Neo4j.Driver.Internal.Types.Node] Key p Value Neo4j.Driver.Internal.Types.Node Id 1 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:1 Labels [ Person ] Properties (empty) 1 [ [p, Neo4j.Driver.Internal.Types.Node] ] Keys [ p ] Values [ [p, Neo4j.Driver.Internal.Types.Node] ] Keys [ p ] Values [ [p, Neo4j.Driver.Internal.Types.Node] ] Keys [ p ] Values [ [p, Neo4j.Driver.Internal.Types.Node] ] Keys [ p ] Values [ [p, Neo4j.Driver.Internal.Types.Node] ] Keys [ p ] Values [ [p, Neo4j.Driver.Internal.Types.Node] ] (values) index value 0 [p, Neo4j.Driver.Internal.Types.Node] (values) index value 0 [p, Neo4j.Driver.Internal.Types.Node] Key p Value Neo4j.Driver.Internal.Types.Node (values) index value 0 [p, Neo4j.Driver.Internal.Types.Node] Key p Value Neo4j.Driver.Internal.Types.Node Id 2 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:2 Labels [ Person ] Properties [ [name, Andreas], [height, 186] ] (values) index value 0 [p, Neo4j.Driver.Internal.Types.Node] Key p Value Neo4j.Driver.Internal.Types.Node Id 2 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:2 Labels [ Person ] Properties key type value name System.String Andreas height System.Int64 186 (values) index value 0 [p, Neo4j.Driver.Internal.Types.Node] Key p Value Neo4j.Driver.Internal.Types.Node Id 2 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:2 Labels [ Person ] Properties key type value name System.String Andreas height System.Int64 186

### Find Nodes with Properties

You can filter nodes by properties by adding the property to the node pattern.

In [22]:
driver.ExecutableQuery("MATCH (andreas:Person {name:'Andreas'}) RETURN andreas").ExecuteAsync().Result.Result

index value 0 [ [andreas, Neo4j.Driver.Internal.Types.Node] ] Keys [ andreas ] Values [ [andreas, Neo4j.Driver.Internal.Types.Node] ] Keys [ andreas ] Values [ [andreas, Neo4j.Driver.Internal.Types.Node] ] Keys [ andreas ] Values [ [andreas, Neo4j.Driver.Internal.Types.Node] ] Keys [ andreas ] Values [ [andreas, Neo4j.Driver.Internal.Types.Node] ] Keys [ andreas ] Values [ [andreas, Neo4j.Driver.Internal.Types.Node] ] (values) index value 0 [andreas, Neo4j.Driver.Internal.Types.Node] (values) index value 0 [andreas, Neo4j.Driver.Internal.Types.Node] Key andreas Value Neo4j.Driver.Internal.Types.Node (values) index value 0 [andreas, Neo4j.Driver.Internal.Types.Node] Key andreas Value Neo4j.Driver.Internal.Types.Node Id 2 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:2 Labels [ Person ] Properties [ [name, Andreas], [height, 186] ] (values) index value 0 [andreas, Neo4j.Driver.Internal.Types.Node] Key andreas Value Neo4j.Driver.Internal.Types.Node Id 2 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:2 Labels [ Person ] Properties key type value name System.String Andreas height System.Int64 186 (values) index value 0 [andreas, Neo4j.Driver.Internal.Types.Node] Key andreas Value Neo4j.Driver.Internal.Types.Node Id 2 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:2 Labels [ Person ] Properties key type value name System.String Andreas height System.Int64 186 1 [ [andreas, Neo4j.Driver.Internal.Types.Node] ] Keys [ andreas ] Values [ [andreas, Neo4j.Driver.Internal.Types.Node] ] Keys [ andreas ] Values [ [andreas, Neo4j.Driver.Internal.Types.Node] ] Keys [ andreas ] Values [ [andreas, Neo4j.Driver.Internal.Types.Node] ] Keys [ andreas ] Values [ [andreas, Neo4j.Driver.Internal.Types.Node] ] Keys [ andreas ] Values [ [andreas, Neo4j.Driver.Internal.Types.Node] ] (values) index value 0 [andreas, Neo4j.Driver.Internal.Types.Node] (values) index value 0 [andreas, Neo4j.Driver.Internal.Types.Node] Key andreas Value Neo4j.Driver.Internal.Types.Node (values) index value 0 [andreas, Neo4j.Driver.Internal.Types.Node] Key andreas Value Neo4j.Driver.Internal.Types.Node Id 3 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:3 Labels [ Person ] Properties [ [name, Andreas], [height, 186] ] (values) index value 0 [andreas, Neo4j.Driver.Internal.Types.Node] Key andreas Value Neo4j.Driver.Internal.Types.Node Id 3 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:3 Labels [ Person ] Properties key type value name System.String Andreas height System.Int64 186 (values) index value 0 [andreas, Neo4j.Driver.Internal.Types.Node] Key andreas Value Neo4j.Driver.Internal.Types.Node Id 3 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:3 Labels [ Person ] Properties key type value name System.String Andreas height System.Int64 186

### Find Nodes using a `WHERE` clause

The `WHERE` clause allows you to filter results based on conditions.

In [23]:
driver.ExecutableQuery("MATCH (a:Person) WHERE a.name STARTS WITH 'A' RETURN a").ExecuteAsync().Result.Result

index value 0 [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 2 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:2 Labels [ Person ] Properties [ [name, Andreas], [height, 186] ] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 2 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:2 Labels [ Person ] Properties key type value name System.String Andreas height System.Int64 186 (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 2 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:2 Labels [ Person ] Properties key type value name System.String Andreas height System.Int64 186 1 [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 3 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:3 Labels [ Person ] Properties [ [name, Andreas], [height, 186] ] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 3 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:3 Labels [ Person ] Properties key type value name System.String Andreas height System.Int64 186 (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 3 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:3 Labels [ Person ] Properties key type value name System.String Andreas height System.Int64 186

## Upserting data

If you re-run the `CREATE` statements from above, you'll create potentially duplicate data. To avoid this, you can use the `MERGE` clause, which acts like an "upsert" (update or insert) operation.

`MERGE` happens in two phases:

1. `MATCH` phase: The query tries to find a node that matches the pattern
2. `CREATE` phase: If no node is found, a new node is created that matches the pattern

The matching works with an exact match of the criteria in the pattern. 

In [25]:
driver.ExecutableQuery("MERGE (a:Person {name:'Andreas'}) RETURN a").ExecuteAsync().Result.Result

index value 0 [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 2 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:2 Labels [ Person ] Properties [ [name, Andreas], [height, 186] ] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 2 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:2 Labels [ Person ] Properties key type value name System.String Andreas height System.Int64 186 (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 2 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:2 Labels [ Person ] Properties key type value name System.String Andreas height System.Int64 186 1 [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 3 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:3 Labels [ Person ] Properties [ [name, Andreas], [height, 186] ] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 3 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:3 Labels [ Person ] Properties key type value name System.String Andreas height System.Int64 186 (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 3 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:3 Labels [ Person ] Properties key type value name System.String Andreas height System.Int64 186

### Set values on `MERGE`

You can use the `SET` clause to set properties on the node found or creating by `MERGE`.  

In [27]:
driver.ExecutableQuery("""
  MERGE (a:Person {name:'Andreas'}) 
  SET a.height = 190
  RETURN a
  """
).ExecuteAsync().Result.Result

index value 0 [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 2 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:2 Labels [ Person ] Properties [ [name, Andreas], [height, 190] ] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 2 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:2 Labels [ Person ] Properties key type value name System.String Andreas height System.Int64 190 (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 2 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:2 Labels [ Person ] Properties key type value name System.String Andreas height System.Int64 190 1 [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 3 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:3 Labels [ Person ] Properties [ [name, Andreas], [height, 190] ] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 3 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:3 Labels [ Person ] Properties key type value name System.String Andreas height System.Int64 190 (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 3 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:3 Labels [ Person ] Properties key type value name System.String Andreas height System.Int64 190

### Set values on `MERGE` with `ON CREATE`

`MERGE` has a special sub-clause that only exectutes when the `CREATE` phase is triggered. This is the `ON CREATE` clause.

To conditionally set properties only when a new node is created, use the `ON CREATE` clause.

This example uses the built-in `datetime()` function to set the `createdAt` property only when the node is created.

In [29]:
driver.ExecutableQuery("""
  MERGE (a:Person {name:'Nigel'}) 
  ON CREATE SET a.createdAt = datetime()
  RETURN a
  """
).ExecuteAsync().Result.Result

index value 0 [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 6 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:6 Labels [ Person ] Properties [ [createdAt, 2024-04-29T13:10:14.922000000Z], [name, Nigel] ] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 6 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:6 Labels [ Person ] Properties key type value createdAt Neo4j.Driver.ZonedDateTime 2024-04-29T13:10:14.922000000Z name System.String Nigel (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 6 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:6 Labels [ Person ] Properties key type value createdAt Neo4j.Driver.ZonedDateTime 2024-04-29T13:10:14.922000000Z UnknownZoneInfo False Reason None Ambiguous False UtcSeconds 1714396214 EpochTicks 17143962149220000 Zone Z LocalDateTime 4/29/2024 1:10:14 PM UtcDateTime 4/29/2024 1:10:14 PM OffsetSeconds 0 Year 2024 Month 4 Day 29 Hour 13 Minute 10 Second 14 Nanosecond 922000000 name System.String Nigel

## Set values on `MERGE` with `ON MATCH`

Similarly, `MERGE` also has an `ON MATCH` sub-clause that only executes when the `MATCH` phase is triggered.

In [31]:
driver.ExecutableQuery("""
  MERGE (a:Person {name:'Nigel'}) 
  ON MATCH SET a.lastRead = datetime()
  RETURN a
  """
).ExecuteAsync().Result.Result

index value 0 [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] Keys [ a ] Values [ [a, Neo4j.Driver.Internal.Types.Node] ] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 6 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:6 Labels [ Person ] Properties [ [createdAt, 2024-04-29T13:10:14.922000000Z], [name, Nigel], [lastRead, 2024-04-29T13:11:56.013000000Z] ] (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 6 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:6 Labels [ Person ] Properties key type value createdAt Neo4j.Driver.ZonedDateTime 2024-04-29T13:10:14.922000000Z name System.String Nigel lastRead Neo4j.Driver.ZonedDateTime 2024-04-29T13:11:56.013000000Z (values) index value 0 [a, Neo4j.Driver.Internal.Types.Node] Key a Value Neo4j.Driver.Internal.Types.Node Id 6 ElementId 4:9b52f2f1-e63c-44a2-9183-eef64f1743e9:6 Labels [ Person ] Properties key type value createdAt Neo4j.Driver.ZonedDateTime 2024-04-29T13:10:14.922000000Z UnknownZoneInfo False Reason None Ambiguous False UtcSeconds 1714396214 EpochTicks 17143962149220000 Zone Z LocalDateTime 4/29/2024 1:10:14 PM UtcDateTime 4/29/2024 1:10:14 PM OffsetSeconds 0 Year 2024 Month 4 Day 29 Hour 13 Minute 10 Second 14 Nanosecond 922000000 name System.String Nigel lastRead Neo4j.Driver.ZonedDateTime 2024-04-29T13:11:56.013000000Z UnknownZoneInfo False Reason None Ambiguous False UtcSeconds 1714396316 EpochTicks 17143963160130000 Zone Z LocalDateTime 4/29/2024 1:11:56 PM UtcDateTime 4/29/2024 1:11:56 PM OffsetSeconds 0 Year 2024 Month 4 Day 29 Hour 13 Minute 11 Second 56 Nanosecond 13000000